# CNN Modeline Dataset Verme Süreci

## 1. Dataset Formatını Seç
Roboflow’dan dataset indirirken birkaç seçenek çıkar:
- **Object Detection için (YOLO, COCO, Pascal VOC)** → bounding box’lı veriler.
- **Classification için (Image Classification)** → kırpılmış nesneler, klasörlere ayrılmış halde.

👉 Sen kırptıysan ve her görüntü tek sınıf içeriyorsa → **Classification dataset** mantıklı olur.  

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset
from torch.nn.utils import clip_grad_norm_
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
import numpy as np
import random
import copy
import os
import torch.optim as optim
from collections import Counter
from tqdm import tqdm
from torchvision import transforms
import time

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # 1 kanal yap
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),  # 1 kanal normalize
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3), value=0)
])

In [ ]:
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # 1 kanal yap
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 1 kanal normalize
])

In [ ]:
train_dataset = datasets.ImageFolder(root=r"C:\Users\hdgn5\OneDrive\Masaüstü\PyTorch CNN Anlatımları\Dataset\- Brain Tümor\Training", transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # <-- buraya ekle
    train_transform
]))
tes_dataset = datasets.ImageFolder(root=r"C:\Users\hdgn5\OneDrive\Masaüstü\PyTorch CNN Anlatımları\Dataset\- Brain Tümor\Testing", transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # <-- buraya ekle
    test_transform
]))

In [ ]:

train_loader = DataLoader(train_dataset,batch_size=32 , shuffle=True)
test_loader = DataLoader(tes_dataset,batch_size=32,shuffle=False)

In [ ]:
for images , label in train_loader:
    print("Batch : " , images.shape )
    print("Label : " , label)
    break

In [ ]:
# labels_test = [label for _ , label in tes_dataset]
# counts_txt = Counter(labels_test)

# for idx , cls in enumerate(tes_dataset.classes):
#     print(f"{cls} : {counts_txt[idx]} örnek")

In [ ]:
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

# 1 batch al
images, labels = next(iter(train_loader))

# 16 örneği grid olarak göster
grid_img = make_grid(images[:16], nrow=4, normalize=True)
plt.figure(figsize=(8,8))
plt.imshow(grid_img.permute(1,2,0))
plt.axis('off')
plt.show()

print(f"Batch boyutu: {images.size(0)}")
print(f"Dataset boyutu (orijinal): {len(train_dataset)}")

In [ ]:
class SeBlock(nn.Module):
    def __init__(self, channels , reduction = 8 ):
        super().__init__()
        self.fc1 = nn.Linear(channels,channels// reduction)
        self.fc2 = nn.Linear(channels // reduction,channels)

    def forward(self,x):
        b,c,_,_ = x.size()
        y = torch.mean(x,dim=(2,3))
        y = F.silu(self.fc1(y))
        y = torch.sigmoid(self.fc2(y)).view(b,c,1,1)
        return x * y 
class MiniBottleNeckSe(nn.Module):
    def __init__(self, in_channels, out_channels , stride=1,use_projection = False , p_drop =0.05):
        super().__init__()
        mid_channels = max(1,out_channels // 4)
        self.p_drop = p_drop

        self.layers = nn.ModuleList()
        chanells = [in_channels , mid_channels , mid_channels ,out_channels]
        kernel_sizes = [1,3,1]
        strides = [1,stride,1]

        for i in range(3):
            self.layers.append(nn.BatchNorm2d(chanells[i]))
            self.layers.append(nn.SiLU())
            self.layers.append(nn.Conv2d(chanells[i] , chanells[i+1] , kernel_size=kernel_sizes[i] , stride=strides[i] , padding= 1 if kernel_sizes[i] == 3 else 0))

        if use_projection or in_channels != out_channels or stride!=1:
            self.shortcut = nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=stride)
        else :
            self.shortcut = nn.Identity()
        
        self.se = SeBlock(out_channels,reduction=8)
    
    def forward(self,x):
        identity = self.shortcut(x)
        out = x

        for i in range(0,len(self.layers),3):
            out = self.layers[i](out)
            out = self.layers[i+1](out)
            out = self.layers[i+2](out)

        if self.training and torch.rand(1).item() < self.p_drop:
            out = identity
        
        out = self.se(out)
        out += identity
        return out
class MiniCNNSe(nn.Module):
    def __init__(self, input_channels = 1, num_classes = 4 , conv_channels =[64,128,256] , r_b = 1):
        super().__init__()

        self.conv_layers = nn.ModuleList()
        self.res_block = nn.ModuleList()
        in_ch = input_channels

        for out_ch in conv_channels:
            self.conv_layers.append(nn.Conv2d(in_ch,out_ch ,kernel_size=3 , stride=1 , padding=1))
            self.conv_layers.append(nn.BatchNorm2d(out_ch))

            for _ in range(r_b):
                stride_val = 2 if in_ch != out_ch else 1 
                use_prof = True if in_ch !=out_ch else False
                self.res_block.append(MiniBottleNeckSe(out_ch ,out_ch , stride=stride_val , use_projection=use_prof))
            
            in_ch = out_ch

            self.global_pool = nn.AdaptiveAvgPool2d((1,1))
            self.fc = nn.Linear(conv_channels[-1] ,  num_classes)
    
    def forward(self,x):
        res_idx = 0
        for i in range(0,len(self.conv_layers),2):
            x = F.silu(self.conv_layers[i+1](self.conv_layers[i](x)))
            x = F.dropout2d(x,p=0.1 , training=self.training)

            for _ in range(1):
                x = self.res_block[res_idx](x)
                res_idx += 1

        x = self.global_pool(x)
        x = torch.flatten(x,1)
        x = self.fc(x)
        return x 

In [ ]:
num_classes = len(train_dataset.classes) 
model = MiniCNNSe(input_channels=1, num_classes=num_classes).to(device)

In [ ]:
from torchinfo import summary
summary(model, input_size=(32,1,224,224), verbose=2)

In [ ]:
loss_fonk = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.Adam(model.parameters() , lr=0.001 , weight_decay=1e-4 , betas=(0.9,0.999) , eps=1e-7)
lr_scheduler = ReduceLROnPlateau(optimizer,mode="min" , patience=5,factor=0.3)  

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, delta=0, verbose=False, restore_best_weights=True):
        self.patience = patience
        self.delta = delta
        self.verbose = verbose
        self.counter = 0
        self.rest_best_weights = restore_best_weights
        self.best_loss = None
        self.early_stop = False
        self.best_model_wts = None

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            if self.rest_best_weights:
                self.best_model_wts = copy.deepcopy(model.state_dict())
        
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
            if self.rest_best_weights:
                self.best_model_wts = copy.deepcopy(model.state_dict())
            if self.verbose:
                print(f"Doğrulama kaybı iyileşti: {val_loss:.4f}. Modelin en iyi ağırlıkları kaydedildi.")
        else:
            self.counter += 1
            if self.verbose:
                print(f"Doğrulama kaybında iyileşme yok. Sayaç: {self.counter}/{self.patience}")
            
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print("Erken durdurma tetiklendi. Eğitim durduruldu.")

    def restore_weights(self, model):
        if self.rest_best_weights and self.best_model_wts is not None:
            model.load_state_dict(self.best_model_wts)

In [ ]:
early_stopping = EarlyStopping(patience=7, verbose=True, restore_best_weights=True)

epochs_num = 10
best_val_loss = float('inf')
max_norm = 2

for epoch in range(epochs_num):
    start_time = time.time()
    model.train()
    train_loss = 0.0
    correct_train = 0.0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs_num}", leave=False):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fonk(outputs, labels)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
        optimizer.step()

        train_loss += loss.item()
        correct_train += (outputs.argmax(1) == labels).sum().item()

    train_loss /= len(train_loader)
    train_acc = 100 * correct_train / len(train_dataset)

     # EVAL
    model.eval()
    val_loss = 0
    correct_val = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_fonk(outputs, labels)
            val_loss += loss.item()
            correct_val += (outputs.argmax(1) == labels).sum().item()

    val_loss /= len(test_loader)
    val_acc = 100 * correct_val / len(tes_dataset)

    lr_scheduler.step(val_loss)

    epoch_time = time.time() - start_time
    print(f"Epoch {epoch+1}/{epochs_num} | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}% | "
          f"Time: {epoch_time:.1f}s | "
          f"LR: {optimizer.param_groups[0]['lr']:.6f}")

     # EarlyStopping çağır
    early_stopping(val_loss, model)

    # Best model kaydet
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")

    if early_stopping.early_stop:
        print("Early stopping triggered!")
        break

early_stopping.restore_weights(model)

In [ ]:
# Modeli eval moduna al

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import torch

model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Confusion Matrix oluştur
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=tes_dataset.classes)

# CM'yi görselleştir
plt.figure(figsize=(8,8))
disp.plot(cmap=plt.cm.Blues, values_format='d')
plt.title("Test Seti Confusion Matrix")
plt.show()

In [ ]:
from sklearn.metrics import classification_report
import torch

model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Sınıf raporu
report = classification_report(all_labels, all_preds, target_names=tes_dataset.classes)
print("Classification Report:\n")
print(report)

# Bilgisayarla Görü Projesi Genel İş Akışı

## 1. Veri Toplama
- Kameradan, internetten, açık datasetlerden veya kendi çektiğin görüntülerden veri toplarsın.  
- Veri çeşitliliği önemlidir (farklı ışık, açı, ortam).



## 2. Veri Etiketleme
- Roboflow, LabelImg, CVAT gibi araçlarla nesneleri işaretlersin.  
- İki ana senaryo vardır:
  - **Image Classification (Sınıflandırma):** Her resim tek bir sınıfa ait (örn. “kedi” veya “köpek”).  
  - **Object Detection (Nesne Tespiti):** Resimde birden çok nesne olabilir, bounding box ile işaretlenir (örn. yol, tümsek, araba).  
  - **Segmentation (Bölütleme):** Piksel bazlı işaretleme (örn. tüm yol pikselleri).  



## 3. Veri Ön İşleme (Preprocessing)
- Görüntüleri boyutlandırma, normalize etme, gürültü temizleme.  
- Augmentation: Döndürme, kırpma, parlaklık değiştirme → modelin genelleme kabiliyetini artırır.  


## 4. Dataset Hazırlama
- Veriyi **train / validation / test** olarak ayır:  
  - %70 → Eğitim  
  - %20 → Doğrulama  
  - %10 → Test  


## 5. Model Seçimi
- **CNN tabanlı modeller:** Image classification için uygundur.  
- **YOLO, Faster R-CNN, SSD:** Object detection için.  
- **U-Net, Mask R-CNN:** Segmentation için.  


## 6. Modelin Eğitilmesi
- Framework seç: **PyTorch, TensorFlow, Keras, Ultralytics YOLO**.  
- Hyperparametreler: batch size, learning rate, epoch sayısı.  
- GPU/TPU varsa hız kazanırsın.  


## 7. Modelin Değerlendirilmesi
- **Accuracy, Precision, Recall, F1-score, mAP** gibi metriklerle başarı ölçülür.  
- Overfitting var mı diye validation loss’a bakılır.  


## 8. Modelin İyileştirilmesi
- Daha fazla veri topla.  
- Augmentation çeşitliliğini artır.  
- Daha güçlü bir model mimarisi dene.  
- Transfer learning kullan (önceden eğitilmiş ağırlıklar).  



## 9. Modelin Kullanıma Alınması (Deployment)
- Eğitimden sonra model şu ortamlara alınabilir:
  - **Edge cihaz:** Raspberry Pi, Jetson Nano (gömülü sistemler).  
  - **Mobil uygulama:** TensorFlow Lite, CoreML.  
  - **Sunucu & API:** Flask, FastAPI, Roboflow API.  

## 10. Gerçek Zamanlı Kullanım
- Kamera akışını al → modele gönder → çıktı (ör. sesli uyarı, görsel işaret, API cevabı).  
- Uygulamanın asıl değer kattığı kısım burasıdır.
